In [1]:
import torch
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
import datasets

# Load the pre-trained model and tokenizer
model_name = "xlm-roberta-large"
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

# Create a sample DataFrame
data = pd.DataFrame({
    'text': [
        'This is a positive example',
        'This is a negative example',
        'Another positive example',
        'Another negative example'
    ],
    'label': [1, 0, 1, 0]
})

# Split the data into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Define a function to preprocess the input text
def preprocess(examples):
    texts = examples['text']
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512)
    labels = examples['label']
    encodings['labels'] = labels
    return encodings

# Prepare the datasets
train_dataset = datasets.Dataset.from_pandas(train_data)
train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = datasets.Dataset.from_pandas(val_data)
val_dataset = val_dataset.map(preprocess, batched=True)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to='none'
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Fine-tune the model
trainer.train()



2024-05-03 08:20:18.850436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 08:20:18.850559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 08:20:18.998499: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.844955
2,No log,0.892491
3,No log,0.925010


TrainOutput(global_step=6, training_loss=0.7538442611694336, metrics={'train_runtime': 94.1044, 'train_samples_per_second': 0.096, 'train_steps_per_second': 0.064, 'total_flos': 114671241972.0, 'train_loss': 0.7538442611694336, 'epoch': 3.0})

In [2]:
# Run inference on the test data
import torch
test_data = pd.DataFrame({'text': ['This is a test example', 'Another test example']})

def preprocess_test(examples):
    texts = examples['text']
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
    return encodings

test_dataset = datasets.Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(preprocess_test, batched=True)

outputs = trainer.predict(test_dataset)
print(outputs.predictions)
preds = torch.softmax(torch.tensor(outputs.predictions), dim=1)[:, 1].tolist()

# Add the predictions to the DataFrame
test_data['probabilities'] = preds

# Print the DataFrame
print(test_data)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

[[-0.10048532  0.18336631]
 [-0.09185255  0.1964754 ]]
                     text  probabilities
0  This is a test example       0.570490
1    Another test example       0.571587
